In [1]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.data import wine_data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn import model_selection
import json
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
import random
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from multiprocessing import Pool
from collections import Counter

In [ ]:
def class_fun(feature,profile,group,test_profile,test_group):#
    exp_pre=profile.loc[feature,:]
    test_exp_pre=test_profile.loc[feature,]
    dat2=exp_pre.stack()
    exp=dat2.unstack(0)
    dat3=test_exp_pre.stack()
    test_exp=dat3.unstack(0)
    rf_new = RandomForestClassifier()
    rf_new.fit(exp, group)
    #prediction = rf_new.predict(test_exp)
    #交叉验证，5次准确率
    scores = cross_val_score(rf_new, exp, group, cv=5,scoring='accuracy')
    #print(scores)
    accuracy_value=pd.DataFrame({'accuracy':scores})
    mean_acc=accuracy_value.loc[:,'accuracy'].mean()
    #print(mean_acc)
    #accuracy_selection.append(accuracy_score(prediction, test_group))
    #cvs_selection.append(np.mean(scores))
    ##train_roc
    y_score = rf_new.predict_proba(exp)[:,1]
    fpr,tpr,threshold = metrics.roc_curve(group, y_score)
    roc_auc = metrics.auc(fpr,tpr)
    ##test_roc
    y_score = rf_new.predict_proba(test_exp)[:,1]
    fpr,tpr,threshold = metrics.roc_curve(test_group, y_score)
    roc_auc_test = metrics.auc(fpr,tpr)

    #print(pred_y_rf_new)
    importance=rf_new.feature_importances_
    data = { 'gene' : feature, 'importance' :importance }
    df = pd.DataFrame(data)
    df.loc['ROC']=['ROC',roc_auc]
    df.loc['ROC_test']=['ROC_test',roc_auc_test]
    df.loc['Mean_accuracy']=['Mean_accuracy',mean_acc]

    return df

# In[9]:


def jurge_fun(x):
    if x[7]==x[10]:
        y="a"
    else:
        y="b"
    return y

def findgene_fun(df1,df2):
    module_res=pd.merge(df1,df2,on='gene')
    module_res.loc[:,'type'] = module_res.apply(jurge_fun,1)
    module_res.loc[:,'id'] = module_res.index
    module_res1 =module_res.loc[module_res['type']=='a']
    rank_accuracy=module_res1.groupby(['modeuleType']).apply(lambda t: t[(t['Mean_accuracy']==t['Mean_accuracy'].max())])
    rank_accuracy1=rank_accuracy.drop(columns=['modeuleType'])
    rank_accuracy1.reset_index(inplace=True)
    rank_impor=rank_accuracy1.groupby(['modeuleType']).apply(lambda t: t[(t['Rank']==t['Rank'].max())])
    rank_impor1=rank_impor.drop(columns=['modeuleType','level_1'])
    rank_impor1.reset_index(inplace=True)
    #res_last=pd.merge(module_res,rank_impor1,on='id',how='right')
    #print(res_last)
    res_last=rank_impor1.loc[:,['gene_pair','modeuleType']]
    return res_last.drop_duplicates()

def combination_fun(x,profile,group,test_profile,test_group):
    #gene_all=[]
    all_value=pd.DataFrame({'gene_pair':[],'gene':[],'importance':[],'Rank':[],'roc':[],'roc_test':[],'module_two_genes':[],
                            'iteration1':[],'iteration2':[]})
    module=x.iloc[:,1].drop_duplicates().values.tolist()
    for mm in module:
        #print(mm)
        gene_module=x.loc[x['modeuleType'] != mm].gene.tolist()
        #print(gene_module)
        y=x[x.modeuleType.isin([mm])].gene
        for i in range(0,len(y)-1):
            for j in range(i+1,len(y)):
                gene=[y[i],y[j]]
                gene_all=gene_module+gene
                #print(gene_all)
                class_results=class_fun(gene_all,profile,group,test_profile,test_group)#
                roc=class_results.loc['ROC'].importance
                roc_test=class_results.loc['ROC_test'].importance
                mean_acc=class_results.loc['Mean_accuracy'].importance
                gene_importance=class_results.drop(index=["ROC","ROC_test","Mean_accuracy"])
                gene_importance['Rank'] = gene_importance['importance'].rank(method="average")

                #gene_all=gene_all.append(gene_importance)
                #roc_all.extend([roc])
                valuedf=pd.DataFrame({'gene_pair':'_'.join(gene),'gene':gene_importance.gene,'importance':gene_importance.importance,
                                      'Rank':gene_importance.Rank,
                                          'Mean_accuracy':mean_acc,
                                          'roc':roc,'roc_test':roc_test,'module_two_genes':mm
                                          ,'iteration1':i,'iteration2':j})
                all_value=pd.concat([valuedf,all_value])
    gene_all=findgene_fun(all_value,x)
                #print(gene)
    all_value.to_csv('all_value_com_moremodule.csv')
#print(gene_all)
    return gene_all





# In[ ]:
gene_module=pd.DataFrame(pd.read_csv('dm_inter_gene_module_4dataset.csv'
                                      ,header=0,index_col=0))

#read data
dat=pd.DataFrame(pd.read_csv('dm_inter_4dataset.csv',header=0,index_col=0))
group=[ 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0]

dat1=pd.DataFrame(pd.read_csv('dm_inter_1dataset.csv',header=0,index_col=0))
####group
y_test=[1, 1, 0, 1, 0, 1, 1, 0]


# gene_last=[]
# for i in range(1,100):
#     print(i)
#     gene_all=combination_fun(gene_module,dat,group,dat1,y_test)
    #gene_all=findgene_fun(tt,gene_module)
#     gene_last.append(gene_all)


arg_l = [(gene_module, dat, group, dat1, y_test) for i in range(1000)]

with Pool(48) as pool:
    res=pool.starmap(combination_fun, arg_l)
    pool.close()
    pool.join()

    
def flat(a):
    l=pd.DataFrame({'gene_pair':[],'modeuleType':[]})
    for i in a:
        l=pd.concat([l,i])
    return(l)

gene_list = flat(res)
print(gene_list)
type(gene_list)


tt=gene_list.groupby('modeuleType').value_counts()
#print(type(tt))
tt=tt.reset_index()
tt.columns = ['moduleType','gene_pair','count']
tt.to_csv('gene_count_all_test.csv')



In [7]:
gene_count_all=pd.DataFrame(pd.read_csv('gene_count_all_test.csv',header=0,index_col=0))
gene_count_all.sort_values(by="count",ascending=False,inplace=True)

In [8]:
print(gene_count_all.groupby('moduleType').head(5))

   moduleType       gene_pair  count
0   MEMagenta     NFIC_PIK3CB     96
2   MEMagenta      NR1D2_KRAS     96
3   MEMagenta  SLC35E2A_UBE2W     96
4   MEMagenta       MBP_UBE2W     96
27      METan    VPREB3_IRAK3     96
28      METan     CD79A_IRAK3     96
29      METan     BANK1_IRAK3     96
1   MEMagenta    SCAF11_UBE2W     96
30      METan     MYOM2_IRAK3     96
31      METan      CD79A_DSC1     71


In [9]:
gene_all=['NFIC','NR1D2','KRAS','IRAK3','SLC35E2A','UBE2W','MBP','VPREB3','BANK1','CD79A','MYOM2','SCAF11','PIK3CB']#gene_count_all.sort_values(by="count",ascending=False)

In [13]:

test=class_fun(gene_all,dat,group,dat1,y_test)

print(test)

                        gene  importance
0                       NFIC    0.088435
1                      NR1D2    0.067023
2                       KRAS    0.121110
3                      IRAK3    0.083662
4                   SLC35E2A    0.032224
5                      UBE2W    0.155011
6                        MBP    0.035187
7                     VPREB3    0.135745
8                      BANK1    0.064532
9                      CD79A    0.056023
10                     MYOM2    0.045686
11                    SCAF11    0.060822
12                    PIK3CB    0.054540
ROC                      ROC    1.000000
ROC_test            ROC_test    0.800000
Mean_accuracy  Mean_accuracy    0.852381


In [14]:
print(test)

                        gene  importance
0                       NFIC    0.088435
1                      NR1D2    0.067023
2                       KRAS    0.121110
3                      IRAK3    0.083662
4                   SLC35E2A    0.032224
5                      UBE2W    0.155011
6                        MBP    0.035187
7                     VPREB3    0.135745
8                      BANK1    0.064532
9                      CD79A    0.056023
10                     MYOM2    0.045686
11                    SCAF11    0.060822
12                    PIK3CB    0.054540
ROC                      ROC    1.000000
ROC_test            ROC_test    0.800000
Mean_accuracy  Mean_accuracy    0.852381


In [15]:
feature=gene_all
profile=dat
group=group
test_profile=dat1
test_group=y_test

In [35]:
exp_pre=profile.loc[feature,:]
test_exp_pre=test_profile.loc[feature,]
dat2=exp_pre.stack()
exp=dat2.unstack(0)
dat3=test_exp_pre.stack()
test_exp=dat3.unstack(0)
rf_new = RandomForestClassifier(random_state= 69)#random_state= 2
rf_new.fit(exp, group)
    

RandomForestClassifier(random_state=69)

In [36]:
scores = cross_val_score(rf_new, exp, group, cv=5,scoring='accuracy')
    #print(scores)
accuracy_value=pd.DataFrame({'accuracy':scores})
mean_acc=accuracy_value.loc[:,'accuracy'].mean()
print(mean_acc)
    #accuracy_selection.append(accuracy_score(prediction, test_group))
    #cvs_selection.append(np.mean(scores))
    ##train_roc
y_score = rf_new.predict_proba(exp)[:,1]
fpr,tpr,threshold = metrics.roc_curve(group, y_score)
roc_auc = metrics.auc(fpr,tpr)
    ##test_roc
y_score = rf_new.predict_proba(test_exp)[:,1]
fpr,tpr,threshold = metrics.roc_curve(test_group, y_score)
roc_auc_test = metrics.auc(fpr,tpr)
    
    #print(pred_y_rf_new)
importance=rf_new.feature_importances_
data = { 'gene' : feature, 'importance' :importance }
df = pd.DataFrame(data)
df.loc['ROC']=['ROC',roc_auc]
df.loc['ROC_test']=['ROC_test',roc_auc_test]
df.loc['Mean_accuracy']=['Mean_accuracy',mean_acc]
print(df)

0.8523809523809524
                        gene  importance
0                       NFIC    0.111005
1                      NR1D2    0.092613
2                       KRAS    0.141262
3                      IRAK3    0.101746
4                   SLC35E2A    0.027389
5                      UBE2W    0.083128
6                        MBP    0.035884
7                     VPREB3    0.148208
8                      BANK1    0.059694
9                      CD79A    0.043378
10                     MYOM2    0.042624
11                    SCAF11    0.057077
12                    PIK3CB    0.055993
ROC                      ROC    1.000000
ROC_test            ROC_test    0.800000
Mean_accuracy  Mean_accuracy    0.852381


In [50]:
y_score

array([0.35, 0.64, 0.13, 0.26, 0.09, 0.14, 0.95, 0.47])

In [51]:
test_group

[1, 1, 0, 1, 0, 1, 1, 0]

In [41]:
scoring = {'accuracy': make_scorer(accuracy_score),
               'roc_auc': make_scorer(roc_auc_score),
               'precision': make_scorer(precision_score),
               'recall': make_scorer(recall_score),
               'f1_score': make_scorer(f1_score)}
rf_cv_score = model_selection.cross_validate(rf_new, exp, group, cv=5, scoring=scoring)
print(rf_cv_score)


{'fit_time': array([0.11259508, 0.08313107, 0.08298564, 0.08201933, 0.07979059]), 'score_time': array([0.01034331, 0.01008415, 0.0096674 , 0.01069355, 0.00961709]), 'test_accuracy': array([0.71428571, 0.85714286, 0.85714286, 1.        , 0.83333333]), 'test_roc_auc': array([0.75      , 0.83333333, 0.83333333, 1.        , 0.83333333]), 'test_precision': array([1. , 0.8, 1. , 1. , 1. ]), 'test_recall': array([0.5       , 1.        , 0.66666667, 1.        , 0.66666667]), 'test_f1_score': array([0.66666667, 0.88888889, 0.8       , 1.        , 0.8       ])}


In [43]:
acc=(0.71428571+ 0.85714286+ 0.85714286+ 1.+ 0.83333333)/5
print(acc)
roc_auc=(0.75   + 0.83333333+0.83333333+ 1.+ 0.83333333)/5
print(roc_auc)
preci=(1.+ 0.8+ 1. + 1.+ 1.)/5
print(preci)
recall=(0.5     + 1.+ 0.66666667+ 1.+ 0.66666667)/5
print(recall)
f1=(0.66666667+ 0.88888889+ 0.8  + 1.  + 0.8        )/5
print(f1)

0.8523809520000001
0.849999998
0.96
0.766666668
0.831111112


In [48]:
data = { 'gene' : featurerank, 'importance' :im_value }
df = pd.DataFrame(data)
df.tail()
#df.iloc[305::]['gene']
df.to_csv('deg_module__inter_importance_dataset4.csv')
df[df['importance']>0]

,gene,importance
0,SLC35E2A,0.027389
1,MBP,0.035884
2,MYOM2,0.042624
3,CD79A,0.043378
4,PIK3CB,0.055993
5,SCAF11,0.057077
6,BANK1,0.059694
7,UBE2W,0.083128
8,NR1D2,0.092613
9,IRAK3,0.101746


In [49]:
print(y_score)
print(y_test)
print(tpr)
print(fpr)
print(roc_auc)


[0.35 0.64 0.13 0.26 0.09 0.14 0.95 0.47]
[1, 1, 0, 1, 0, 1, 1, 0]
[0.  0.2 0.4 0.4 1.  1. ]
[0.         0.         0.         0.33333333 0.33333333 1.        ]
0.8
